<a href="https://colab.research.google.com/github/OneFineStarstuff/Onefinebot/blob/main/Event_Detection_with_Data_Streams_in_Apache_Kafka_and_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade kafka-python

In [ ]:
pip install kafka-python==2.0.2

In [ ]:
pip list

In [ ]:
!python -m venv myenv
!source myenv/bin/activate  # On Windows use `myenv\Scripts\activate`
!pip install kafka-python

In [ ]:
!sudo apt install python3.10-venv

In [ ]:
!python3 -m venv myenv
!source myenv/bin/activate

In [ ]:
!pip install kafka-python
!pip install tensorflow

In [ ]:
from kafka import KafkaConsumer
import tensorflow as tf
import numpy as np

# Kafka consumer to stream astronomical data
consumer = KafkaConsumer(
    'astronomy_data',
    bootstrap_servers=['localhost:9092'],
    value_deserializer=lambda x: np.frombuffer(x, dtype=np.float32)
)

# Placeholder RNN for detecting anomalies in time-series data
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(None, 1), return_sequences=True),
    tf.keras.layers.Dense(1)
])

# Process incoming data stream and detect events
for message in consumer:
    data = message.value.reshape(-1, 1)  # Reshape as required by model
    predictions = model.predict(data)

    if np.any(predictions > 0.5):  # Example threshold for anomaly
        print("Potential event detected!")

In [ ]:
pip install confluent-kafka

In [ ]:
from confluent_kafka import Consumer, KafkaError
import tensorflow as tf
import numpy as np

conf = {'bootstrap.servers': "localhost:9092",
        'group.id': "mygroup",
        'auto.offset.reset': 'earliest'}

consumer = Consumer(conf)
consumer.subscribe(['astronomy_data'])

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(None, 1)),  # Use Input layer
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dense(1)
])

while True:
    msg = consumer.poll(1.0)
    if msg is None:
        continue
    if msg.error():
        if msg.error().code() == KafkaError._PARTITION_EOF:
            continue
        else:
            print(msg.error())
            break

    data = np.frombuffer(msg.value(), dtype=np.float32).reshape(-1, 1)
    predictions = model.predict(data)

    if np.any(predictions > 0.5):  # Example threshold for anomaly
        print("Potential event detected!")

consumer.close()